In [31]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import csv
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


### Download and explore dataset

I am going to recover the rent prices in 2019 on the website <a href="https://www.journaldunet.fr/patrimoine/guide-de-l-immobilier/1181859-loyer-de-reference/">journa du net</a>

In [32]:
requete=requests.get('https://www.journaldunet.fr/patrimoine/guide-de-l-immobilier/1181859-loyer-de-reference/')
page=requete.content
soup=BeautifulSoup(page)
table=soup.find_all('table')
source=['<!doctype html><html><head></head><body>'+str(table[i])+'</body></html>' for i in range(len(table)) ]
for i in range(len(source)):
    soup2=BeautifulSoup(source[i])
    ligne=soup2.find_all('tr')
    ligne=ligne[1:]
    t=[]
    nom='data'+str(i)+'.csv'
    csv_file=open(nom,"w")
    csv_writer=csv.writer(csv_file)
    csv_writer.writerow(['Borough','NbrePieces','NonMeuble','Meuble'])
    if i==0:
        for elem in ligne:
            t=elem.text.split('\n')
            csv_writer.writerow([1,t[0],t[1],t[2],t[3]])
    else:
        for elem in ligne:
            t=elem.text.split('\n')
            csv_writer.writerow([str(i+1)+'ème arrondissement',t[0],t[1],t[2]])
    csv_file.close()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


After recovering neighborhoods

In [33]:
em=soup.find_all('em')
em=em[5:]
csv_file=open("arrond.csv","w")
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Borough','Neighborhood'])
for i in range(len(em)):
    if i==0:
        t=em[i].text.split(': ')
        csv_writer.writerow([str(i+1)+'er arrondissement',t[1]])
    else:
        t=em[i].text.split(': ')
        csv_writer.writerow([str(i+1)+'ème arrondissement',t[1]])
csv_file.close()

In [34]:
df=pd.read_csv('arrond.csv')
df.head()

,Borough,Neighborhood
0,1er arrondissement,"Les Halles, Palais Royal, Place Vendôme et Sai..."
1,2ème arrondissement,"Bonne Nouvelle, Gaillon, Mail, Vivienne."
2,3ème arrondissement,"Arts et Métiers, Enfants Rouges, Les Archives,..."
3,4ème arrondissement,"Arsenal, Notre-Dame, Saint-Gervais, Saint-Merri."
4,5ème arrondissement,"Jardin des Plantes, Saint-Victor, Sorbonne, Va..."


calculate the average of the prices of furnished and unfurnished rents

In [35]:
for i in range(20):
    myfile='data'+str(i)+'.csv'
    vars()['d' + str(i+1)]=pd.read_csv(myfile)
    vars()['d' + str(i+1)]['NonMeuble']=vars()['d' + str(i+1)][['NonMeuble']].apply(lambda x:x.str.replace(',','.'))
    vars()['d' + str(i+1)]['Meuble']=vars()['d' + str(i+1)][['Meuble']].apply(lambda x:x.str.replace(',','.'))
    vars()['d' + str(i+1)]['NonMeuble']=pd.to_numeric(vars()['d' + str(i+1)]['NonMeuble'])
    vars()['d' + str(i+1)]['Meuble']=pd.to_numeric(vars()['d' + str(i+1)]['Meuble'])
    vars()['d' + str(i+1)]['Avg_Meuble']=vars()['d' + str(i+1)]['Meuble'].mean()
    vars()['d' + str(i+1)]['Avg_NonMeuble']=vars()['d' + str(i+1)]['NonMeuble'].mean()
dataPrix=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20],ignore_index=True)

In [36]:
dataPrix.loc[:3,'Borough']='1er arrondissement'
dataPrix.head()

,Borough,NbrePieces,NonMeuble,Meuble,Avg_Meuble,Avg_NonMeuble
0,1er arrondissement,1 pièce,29.83,33.69,28.81,25.4975
1,1er arrondissement,2 pièces,25.75,29.10,28.81,25.4975
2,1er arrondissement,3 pièces,23.21,26.23,28.81,25.4975
3,1er arrondissement,4 pièces et plus,23.20,26.22,28.81,25.4975
4,2ème arrondissement,1 pièce,29.68,33.54,28.62,25.3275


merge the two dataframes (df, dataPrix) and store the result in df1

In [37]:
df1=df.merge(dataPrix[['Borough','Avg_Meuble','Avg_NonMeuble']])
df1.shape

(80, 4)

Let's remember one line per district

In [38]:
df_Prix=df1.loc[::4,]
df_Prix.loc[0,'Neighborhood']="Les Halles, Palais Royal, Place Vendôme,Saint-Germain-l'Auxerrois"
df_Prix.reset_index(drop=True,inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [39]:
df_Prix.head()

,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble
0,1er arrondissement,"Les Halles, Palais Royal, Place Vendôme,Saint-...",28.8100,25.4975
1,2ème arrondissement,"Bonne Nouvelle, Gaillon, Mail, Vivienne.",28.6200,25.3275
2,3ème arrondissement,"Arts et Métiers, Enfants Rouges, Les Archives,...",28.2900,25.0325
3,4ème arrondissement,"Arsenal, Notre-Dame, Saint-Gervais, Saint-Merri.",29.2850,25.9175
4,5ème arrondissement,"Jardin des Plantes, Saint-Victor, Sorbonne, Va...",27.7925,24.5925


I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [41]:
!wget -q -O 'geoCordonnes.csv' https://www.data.gouv.fr/fr/datasets/r/0d3553c6-45c0-4b16-82be-5ef314437d3e
print('Data downloaded!')

Data downloaded!


#### Load and explore the data
Next, let's load the data.

In [93]:
dfParis=pd.read_csv('geoCordonnes.csv',sep=';')
dfParis.head()

,N_SQ_AR,C_AR,C_ARINSEE,L_AR,L_AROFF,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
0,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225, 2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764
1,750000003,3,75103,3ème Ardt,Temple,750001537,1.170883e+06,4519.263648,"48.86287238, 2.3600009859","{""type"": ""Polygon"", ""coordinates"": [[[2.363828...",3,4519.071982
2,750000012,12,75112,12ème Ardt,Reuilly,750001537,1.631478e+07,24089.666298,"48.8349743815, 2.42132490078","{""type"": ""Polygon"", ""coordinates"": [[[2.413879...",12,24088.038922
3,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018, 2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862
4,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263, 2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779


In [94]:
s=list(dfParis['Geometry X Y'].str.split(','))

In [95]:
latitude,longitude=[s[i][0] for i in range(len(s))],[s[i][1] for i in range(len(s))]

In [96]:
latitude=pd.Series(latitude)
longitude=pd.Series(longitude)

In [97]:
dfParis['Latitude']=latitude
dfParis['Longitude']=longitude

In [98]:
dfParis=dfParis[['C_AR','Latitude','Longitude']]
dfParis.sort_values(by='C_AR',inplace=True)
dfParis.reset_index(drop=True,inplace=True)
dfParis.head()

,C_AR,Latitude,Longitude
0,1,48.8625627018,2.33644336205
1,2,48.8682792225,2.34280254689
2,3,48.86287238,2.3600009859
3,4,48.8543414263,2.35762962032
4,5,48.8444431505,2.35071460958


let's merge the two dataframes

In [99]:
paris_data=df_Prix.join(dfParis[['Latitude','Longitude']])

In [100]:
paris_data['Latitude']=pd.to_numeric(paris_data['Latitude'])
paris_data['Longitude']=pd.to_numeric(paris_data['Longitude'])

In [105]:
paris_data.head()

,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble,Latitude,Longitude
0,1er arrondissement,"Les Halles, Palais Royal, Place Vendôme,Saint-...",28.8100,25.4975,48.862563,2.336443
1,2ème arrondissement,"Bonne Nouvelle, Gaillon, Mail, Vivienne.",28.6200,25.3275,48.868279,2.342803
2,3ème arrondissement,"Arts et Métiers, Enfants Rouges, Les Archives,...",28.2900,25.0325,48.862872,2.360001
3,4ème arrondissement,"Arsenal, Notre-Dame, Saint-Gervais, Saint-Merri.",29.2850,25.9175,48.854341,2.357630
4,5ème arrondissement,"Jardin des Plantes, Saint-Victor, Sorbonne, Va...",27.7925,24.5925,48.844443,2.350715


#### Use geopy library to get the latitude and longitude values of Paris.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [101]:
address = 'Paris'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris 48.8566101, 2.3514992.


#### Create a map of Parisk with neighborhoods superimposed on top.

In [104]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Borough'], paris_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 3 clusters.

In [106]:
paris_clustering = paris_data[['Avg_Meuble','Avg_NonMeuble']]
# set number of clusters
kclusters = 3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster

In [107]:
paris_data.insert(0, 'Cluster Labels', kmeans.labels_)
paris_data

,Cluster Labels,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble,Latitude,Longitude
0,1,1er arrondissement,"Les Halles, Palais Royal, Place Vendôme,Saint-...",28.8100,25.4975,48.862563,2.336443
1,1,2ème arrondissement,"Bonne Nouvelle, Gaillon, Mail, Vivienne.",28.6200,25.3275,48.868279,2.342803
2,1,3ème arrondissement,"Arts et Métiers, Enfants Rouges, Les Archives,...",28.2900,25.0325,48.862872,2.360001
3,1,4ème arrondissement,"Arsenal, Notre-Dame, Saint-Gervais, Saint-Merri.",29.2850,25.9175,48.854341,2.357630
4,1,5ème arrondissement,"Jardin des Plantes, Saint-Victor, Sorbonne, Va...",27.7925,24.5925,48.844443,2.350715
5,1,6ème arrondissement,"Monnaie, Notre-Dame-des-Champs, Odéon, Saint-G...",30.0575,26.6025,48.849130,2.332898
6,1,7ème arrondissement,"Ecole Militaire, Gros-Caillou, Invalides, Sain...",31.3775,27.7775,48.856174,2.312188
7,1,8ème arrondissement,"Champs-Elysées, Europe, Faubourg du Roule, Mad...",29.4350,26.0500,48.872721,2.312554
8,0,9ème arrondissement,"Chaussée-d'Antin, Faubourg Montmartre, Rochech...",27.1925,24.0650,48.877164,2.337458
9,0,10ème arrondissement,"Hôpital Saint-Louis, Porte Saint-Denis, Porte ...",25.7575,22.7975,48.876130,2.360728


Finally, let's visualize the resulting clusters

In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Neighborhood'], paris_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

# Cluster 1
neighborhoods where the price of rent is lower

In [117]:
paris_data.loc[paris_data['Cluster Labels'] == 0, paris_data.columns[[1] + list(range(2, paris_data.shape[1]))]]

,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble,Latitude,Longitude
8,9ème arrondissement,"Chaussée-d'Antin, Faubourg Montmartre, Rochech...",27.1925,24.0650,48.877164,2.337458
9,10ème arrondissement,"Hôpital Saint-Louis, Porte Saint-Denis, Porte ...",25.7575,22.7975,48.876130,2.360728
10,11ème arrondissement,"Folie-Méricourt, Roquette, Saint-Ambroise, Sai...",25.7175,22.7625,48.859059,2.380058
12,13ème arrondissement,"Croulebarbe, Gare, Maison Blanche, Salpêtrière.",25.0750,22.1925,48.828388,2.362272
13,14ème arrondissement,"Montparnasse, Parc de Montsouris, Petit Montro...",25.8925,22.9175,48.829245,2.326542
14,15ème arrondissement,"Grenelle, Javel, Necker, Saint-Lambert.",26.7125,23.6450,48.840085,2.292826
16,17ème arrondissement,"Batignolles, Epinettes, Plaine Monceau, Ternes.",26.7550,23.6800,48.887327,2.306777


# Cluster 2
neighborhoods where the rent price is very high

In [118]:
paris_data.loc[paris_data['Cluster Labels'] == 1, paris_data.columns[[1] + list(range(2, paris_data.shape[1]))]]

,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble,Latitude,Longitude
0,1er arrondissement,"Les Halles, Palais Royal, Place Vendôme,Saint-...",28.8100,25.4975,48.862563,2.336443
1,2ème arrondissement,"Bonne Nouvelle, Gaillon, Mail, Vivienne.",28.6200,25.3275,48.868279,2.342803
2,3ème arrondissement,"Arts et Métiers, Enfants Rouges, Les Archives,...",28.2900,25.0325,48.862872,2.360001
3,4ème arrondissement,"Arsenal, Notre-Dame, Saint-Gervais, Saint-Merri.",29.2850,25.9175,48.854341,2.357630
4,5ème arrondissement,"Jardin des Plantes, Saint-Victor, Sorbonne, Va...",27.7925,24.5925,48.844443,2.350715
5,6ème arrondissement,"Monnaie, Notre-Dame-des-Champs, Odéon, Saint-G...",30.0575,26.6025,48.849130,2.332898
6,7ème arrondissement,"Ecole Militaire, Gros-Caillou, Invalides, Sain...",31.3775,27.7775,48.856174,2.312188
7,8ème arrondissement,"Champs-Elysées, Europe, Faubourg du Roule, Mad...",29.4350,26.0500,48.872721,2.312554
15,16ème arrondissement,"Auteil, Chaillot, Muette, Porte Dauphine.",28.3325,25.0800,48.860392,2.261971


# Cluster 3
neighborhoods where the price of rent is the lowest

In [119]:
paris_data.loc[paris_data['Cluster Labels'] == 2, paris_data.columns[[1] + list(range(2, paris_data.shape[1]))]]

,Borough,Neighborhood,Avg_Meuble,Avg_NonMeuble,Latitude,Longitude
11,12ème arrondissement,"Bel Air, Bercy, Picpus, Quinze Vingts.",24.255,21.4675,48.834974,2.421325
17,18ème arrondissement,"Chapelle, Clignancourt, Goutte-d'Or, Grandes C...",23.885,21.1375,48.892569,2.348161
18,19ème arrondissement,"Amérique, Combat, Pont de Flandre, Villette.",22.345,19.7725,48.887076,2.384821
19,20ème arrondissement,"Belleville, Charonne, Père Lachaise, Saint-Far...",23.160,20.5025,48.863461,2.401188
